In [ ]:
import glob
import os
import shutil

from astropy.io import fits
from astropy import stats
from astropy.time import Time
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Range1d
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Determine where the visualization will be rendered
output_notebook()

In [ ]:
postflash_data = pd.read_pickle('/grp/hst/wfc3u/postflash_2021/flt_postflash_with_stats.pkl')

In [ ]:
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
fullframe_pf_A = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'A') & (postflash_data['flash_cur'] == 'MED')] 
fullframe_pf_B = postflash_data.loc[(postflash_data['subarray'] == False) & (postflash_data['shutter'] == 'B') & (postflash_data['flash_cur'] == 'MED')] 

In [ ]:
# normalized to the mean of first observation
sorted_x = fullframe_pf_A['datetime'].sort_values()
first_rootname = sorted_x.index[0]
x = Time(sorted_x,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_A['sig_mean'])
y = fullframe_pf_A['sig_mean']/fullframe_pf_A['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(sorted_x, y,size = 4, legend_label = "Med fullframe Shutter A")
p.line(sorted_x,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)

In [ ]:
# normalized to the mean of first observation
sorted_x = fullframe_pf_B['datetime'].sort_values()
first_rootname = sorted_x.index[0]
x = Time(sorted_x,format='datetime64')
x.format = 'mjd'
x = x.to_value('mjd', 'float')

mean_mean = np.mean(fullframe_pf_B['sig_mean'])
y = fullframe_pf_B['sig_mean']/fullframe_pf_B['sig_mean'][first_rootname]

# Make and fit a linear regression model
def line(x, a, b):
    return a * x + b

#x = x.reshape(-1, 1)
popt, pcov = curve_fit(line, x, list(y))
print(popt[0])

# Find the slope and intercept from the model
slope = popt[0] # Takes the first element of the array
intercept = popt[1]

# Make the regression line
y_pred = []
for i in x:
    i = float(i)
    temp_y = slope*i + intercept 
    y_pred.append(temp_y)

p = figure(title = 'Mean PF value Normalized to Mean of First Observation',x_axis_type='datetime')
p.y_range=Range1d(0.9, 1.05)
p.circle(sorted_x, y,size = 4, legend_label = "Med fullframe Shutter B")
p.line(sorted_x,y_pred,color='red',legend_label='y = ' + str(slope) +' x + '+ str(round(intercept,6)))
show(p)